In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import gdal, gdalnumeric, osr, ogr
import matplotlib.pyplot as plt
import pyproj
from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Image stuff (Getting RGB Values from UTM X,Y)
def getRGBValue(file, x, y):
    band_r = file.GetRasterBand(1)
    band_g = file.GetRasterBand(2)
    band_b = file.GetRasterBand(3)
    
    transform = file.GetGeoTransform()
    
    cols = ds.RasterXSize
    rows = ds.RasterYSize
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    data_r = band_r.ReadAsArray(0, 0, cols, rows)
    data_g = band_g.ReadAsArray(0, 0, cols, rows)
    data_b = band_b.ReadAsArray(0, 0, cols, rows)

    col = int((point[0] - xOrigin) / pixelWidth)
    row = int((yOrigin - point[1] ) / pixelHeight)
    return data_r[row][col], data_g[row][col], data_b[row][col]

In [ ]:
# Transforming Geographic into UTM (nomin data was originally in geographic)
P = pyproj.Proj("+proj=utm +zone=24 +south +ellps=WGS84 +datum=WGS84 +no_defs preserve_units=True")
def LatLon_To_XY(Lat,Lon):
    return P(Lat,Lon)    

def XY_To_LatLon(x,y):
    return P(x,y,inverse=True)  

In [ ]:
# NN Stuff
def norm(x, stats):
  return (x - stats['mean']) / stats['std']

def build_model(data):
  model = keras.Sequential([
    layers.Dense(9, activation='relu', input_shape=[len(data.keys())]),
    layers.Dense(9, activation='relu', input_shape=[len(data.keys())]),
    layers.Dense(1, activation='tanh') # or sigmoid 
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='binary_crossentropy',
                optimizer=optimizer, metrics=['accuracy'])
  return model

EPOCHS = 500

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

In [ ]:
# Mineralizations
points_list = [ (374923.25814789, 8624420.3223381), 
                (372968.03530681, 8621952.0539831),
                (373669.09020306, 8620119.0939694),
                (371987.09636012, 8456312.7980538),
                (371478.32389800, 8458998.2847391),
                (385574.62831851, 8422300.9159796),
                (384376.78480892, 8415583.1654070),
                (383659.85608419, 8412443.8162096),
                (367882.17311751, 8414402.8219878)] #Iron

# No mineralizations
nomin = [(355972.99961472437,8622651.999638285),
(348627.00012491795,8639213.9994279),
(354569.9998716177,8638458.00042752),
(368923.0001300302,8639273.999731105),
(365024.9997199229,8617623.000400761),
(387189.9998726186,8633381.999411695),
(344666.00052456406,8663473.999884512),
(344666.00052456406,8663473.999884512),
(391008.9999675751,8653401.000171833),
(391008.9999675751,8653401.000171833),
(387213.00053199177,8653237.999896513),
(387213.00053199177,8653237.999896513),
(369915.00028302544,8659700.000410985),
(337119.9994985054,8648786.00026137),
(360197.9996720799,8663936.000256341),
(347843.9998121522,8649579.000471167),
(350765.0002621623,8646883.999576418),
(390536.99999982544,8645936.000472099),
(390536.99999982544,8645936.000472099),
(350247.0002481592,8659295.999996332),
(349558.9997809423,8661209.000451935),
(349159.0003225295,8664374.00028235),
(350748.0003647405,8665030.9996883),
(346728.00027293863,8668532.000502877),
(370266.00018438604,8667344.999492563),
(341727.00000418036,8633301.999921042),
(382347.9995332316,8672709.999773592),
(355458.9998121344,8628477.999493144),
(354613.9997159671,8632883.99999678),
(358609.000241934,8635067.000265181),
(369371.9996825262,8629168.999551656),
(389148.9999507639,8647427.00000343),
(337699.99961067596,8622568.000465417),
(349560.0000205182,8625408.000320567),
(388939.1446704907,8403093.256427528),
(383604.9331913097,8401826.835138457),
(382407.51759011677,8401855.464319),
(345124.59553125803,8408571.737873927),
(387344.79782381095,8408315.491347618),
(384388.1715902182,8409968.480568433),
(376301.60677538253,8420013.621377407),
(370856.63516493386,8411002.483419413),
(340410.4973124497,8434863.770088296),
(354664.26822518534,8432232.81937472),
(350903.4918931278,8446944.261651061),
(389519.761529148,8398483.23057417),
(387080.5779655394,8400638.75763532),
(362353.08858660585,8406127.405244118),
(353104.8473335811,8402415.709251491),
(347312.1359313509,8419049.530402021),
(377737.18652860454,8436081.524765661),
(378045.65005886386,8445280.313683847),
(370098.62898758356,8438160.192725942),
(373548.86538814026,8420749.989790335),
(368886.38260180596,8423052.410671575),
(377929.11352431856,8427003.444888312),
(377283.62242520874,8427976.626991184),
(369598.44214613107,8430633.056824991),
(339930.3821094195,8445740.187778326),
(355769.0380139411,8422701.30937814),
(365552.02561771526,8436754.358661316),
(369429.9547960067,8490080.000532107),
(384025.38768577244,8486145.851696257),
(387441.2135341307,8483466.950376488),
(384528.60451368854,8471178.485690035),
(373255.3871814442,8487686.675645975),
(367627.77576319757,8486052.571495354),
(374941.09956260497,8505109.745146638),
(337933.4045829789,8462647.769626757),
(341808.78430278215,8461329.467716811),
(345232.93990633794,8459302.208664259),
(350520.3037301644,8459003.060001615),
(362808.25433477934,8497627.045631679),
(370601.46621792554,8485045.01324625),
(387900.2978840824,8461580.88132007),
(387900.2978840824,8461580.88132007),
(341054.57718033873,8501543.088783683),
(378510.7554454653,8494606.977357453),
(361288.6029805169,8489435.577927772),
(367804.71995707194,8489827.06874375),
(364938.478208927,8464080.950997962),
(364938.478208927,8464080.950997962),
(364938.478208927,8464080.950997962),
(389836.9969546367,8485866.558867155),
(363356.8383568005,8506372.47583962),
(345563.8305896614,8552996.526959116),
(385239.0001430388,8615693.00012989),
(385239.0001430388,8615693.00012989),
(364373.0003930769,8598308.999937812),
(360174.0001582297,8606453.000467332),
(360174.0001582297,8606453.000467332),
(359402.000531684,8609588.999552906),
(342315.99991751433,8616546.000015492),
(342438.9999324347,8568071.999458268),
(347110.0000822337,8572158.00031762),
(350919.9996784302,8580319.999912126),
(345702.9997455516,8592258.99949608),
(343370.99956156116,8598419.00041813),
(371313.9998706489,8568420.000429418),
(380949.0004966053,8614058.999482587),
(385379.00047995814,8601778.000370787),
(385388.99988242186,8604742.000285113),
(373719.0001860682,8584363.9997515),
(391062.30914583505,8560736.00168989),
(377674.28935801185,8556963.440411892),
(341356.1512678982,8530725.99843119),
(339479.75324405916,8532872.111924915),
(388520.6398463956,8529866.908927009),
(373244.6446247114,8518649.734709324),
(365142.4180182029,8519063.670746846),
(354312.4752519874,8523110.948205259),
(355605.39729161974,8518350.22519509),
(358405.2845000661,8523387.261166466),
(354287.5479606765,8513763.392305123),
(383786.9704791966,8542423.040923053),
(371106.1268120706,8542432.31957197),
(374899.82594886224,8537671.640605502),
(379665.5819198127,8533302.009406388),
(338871.4191048284,8511473.25331358),
(338871.4191048284,8511473.25331358),
(374498.3053082531,8528455.867423316),
(365548.244210623,8531034.283831488),
(390483.88556019217,8561651.721125105),
(390483.88556019217,8561651.721125105),
(390483.88556019217,8561651.721125105),
(369218.4868641846,8549579.925489895),
(358564.6491445552,8557314.280903464),
(345178.4890736592,8536832.455003446),
(348047.8104628704,8537291.171281956),
(355477.3666925371,8552519.586517658),
(367442.86954620853,8558066.401501687),
(367354.921805789,8556063.90029429),
(367354.921805789,8556063.90029429),
(367354.921805789,8556063.90029429),
(359870.4680224909,8550075.664748395),
(380857.7044980444,8518551.966670532),
(384823.73535912397,8558763.996982682),
(383604.8187179938,8549556.418444399),
(373666.0001084213,8579871.999771012),
(370171.9999038365,8569187.999419793),
(339495.99995250633,8616470.000470823),
(339495.99995250633,8616470.000470823),
(339037.00020410126,8612255.99982463),
(343735.9995290679,8606400.000441771),
(376308.9996579266,8564196.000301503),
(378249.99985738203,8609961.999875609),
(343314.9995209273,8665476.000293508),
(339818.9994969594,8651174.999883926),
(347557.9997447904,8649821.999850787),
(366757.0001066203,8648197.000276536),
(342626.000438604,8595413.999915658),
(343278.9998268107,8595343.00048659),
(382989.77017877623,8526270.829145718),
(386933.46944061,8528610.442102084),
(384221.99976006825,8604485.999447864),
(374737.0000307679,8585271.999717027),
(386933.933360435,8528499.836698104),
(376924.0003602829,8640936.999417389),
(381483.9998598406,8627050.999508996),
(344666.00052456406,8663473.999884512),
(389539.0004176666,8651737.000033678),
(387213.00053199177,8653237.999896513),
(387213.00053199177,8653237.999896513),
(383079.00007860427,8655868.00031889),
(379244.0004158501,8656160.000344258),
(339818.9994969594,8651174.999883926),
(365996.99952924263,8671533.000445107),
(359780.99992724345,8647223.999535985),
(374579.9994940529,8668514.000197098),
(372867.999667256,8621982.9996364),
(389148.9999507639,8647427.00000343),
(364147.21211550495,8425977.983361017),
(361909.9349206948,8410649.122601988),
(351738.9369990615,8446637.939874776),
(359929.0254739853,8431911.239394028),
(364033.3772299871,8398396.601639353),
(344919.000461554,8593492.000123283),
(339495.99995250633,8616470.000470823),
(350919.9996784302,8580319.999912126),
(341815.9999590209,8595835.999476219),
(360174.0001582297,8606453.000467332),
(375321.9995531054,8615328.99981738),
(344919.000461554,8593492.000123283),
(344919.000461554,8593492.000123283),
(346320.000461071,8605754.000308268),
(347110.0000822337,8572158.00031762),
(370726.07121315313,8488240.784976568),
(359957.9283473515,8505500.560389439),
(387900.2978840824,8461580.88132007),
(387900.2978840824,8461580.88132007),
(384009.00806417596,8464532.400155302),
(384009.00806417596,8464532.400155302),
(384009.00806417596,8464532.400155302),
(355381.74796786986,8488046.676681176),
(377656.99663622025,8482999.975574275),
(387387.63464035955,8560289.884523632),
(368531.47986146365,8519169.015177146),
(338871.4191048284,8511473.25331358),
(390483.88556019217,8561651.721125105),
(390483.88556019217,8561651.721125105),
(365036.5381775177,8553397.901316645),
(342476.7489847214,8548631.49216559),
(342476.7489847214,8548631.49216559),
(344251.8441733975,8547258.880560726),
(384823.73535912397,8558763.996982682),
(337145.87638951984,8648795.797020951),
(337145.87638951984,8648795.797020951),
(337145.87638951984,8648795.797020951),
(337145.87638951984,8648795.797020951),
(337145.87638951984,8648795.797020951),
(337145.87638951984,8648795.797020951),
(381454.01097882964,8406959.986114651),
(381429.9743536998,8406778.013175115),
(381366.02101863554,8406465.982246589),
(381366.02101863554,8406465.982246589),
(381366.02101863554,8406465.982246589),
(381082.29088548175,8423065.95330646),
(381082.29088548175,8423065.95330646),
(381082.29088548175,8423065.95330646),
(381112.4690281326,8422184.802259708),
(381112.4690281326,8422184.802259708),
(381112.4690281326,8422184.802259708),
(380705.95153791585,8421643.054161785),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(386399.0532011617,8430350.980094835),
(380879.4091012465,8423804.361938493),
(380879.4091012465,8423804.361938493),
(380081.1493100712,8402928.927607728),
(378909.6339270385,8403995.580036443),
(377193.24291964003,8404889.024156488),
(377193.24291964003,8404889.024156488),
(382617.1874113051,8412182.179480342),
(381973.31189887243,8413016.298379922),
(381973.31189887243,8413016.298379922),
(380985.83605613234,8413705.403724665),
(388462.22970115725,8434839.648667315),
(387449.4030327523,8435315.339237673),
(388145.67793765693,8429890.55754558),
(385724.3621720663,8430063.868923701),
(382205.73083746363,8430504.84881556),
(391894.500724833,8447683.997586733),
(391894.500724833,8447683.997586733),
(391894.500724833,8447683.997586733),
(391894.500724833,8447683.997586733),
(389059.34809402784,8449869.815222139),
(389059.34809402784,8449869.815222139),
(384457.14462372655,8472369.356212622),
(384528.60451368854,8471178.485690035),
(367882.174875233,8414402.490079511),
(371441.1309183474,8458960.267124731),
(372025.6832353154,8456417.193155855),
(372025.6832353154,8456417.193155855),
(380774.0003288376,8614709.000336953),
(362603.1896657089,8554292.859694833),
(362603.1896657089,8554292.859694833),
(362603.1896657089,8554292.859694833),
(368004.4405473077,8414308.554231543),
(368004.4405473077,8414308.554231543),
(368004.4405473077,8414308.554231543),
(371987.1013287983,8456311.80246662),
(371987.1013287983,8456311.80246662),
(371987.1013287983,8456311.80246662),
(371987.1013287983,8456311.80246662),
(371987.1013287983,8456311.80246662),
(387498.3425680517,8429784.029312767),
(387178.5729091568,8428848.783982908),
(384376.7853213253,8415583.05474503),
(384326.15695247444,8414848.09811157),
(385574.6323569982,8422300.031018876),
(383659.8586672344,8412443.263084512),
(374923.2581478642,8624420.322297078),
(374891.46588707203,8622876.262152756),
(371225.7782562519,8457930.104041616),
(367066.38170248264,8409266.41282938)]



df_columns = [ 'GT_r', 'GT_g', 'GT_b',
          'eTh_r', 'eTh_g', 'eTh_b', 
          'eU_r', 'eU_g', 'eU_b',
          'KPERC_r', 'KPERC_g', 'KPERC_b',
          'ThK_r', 'ThK_g', 'ThK_b',
          'UK_r', 'UK_g', 'UK_b',
          'UTh_r', 'UTh_g', 'UTh_b',]
files = [ 'mag_GT.tif', 'gama_eTh.tif',  'gama_eU.tif', 'gama_KPERC.tif', 'gama_ThK.tif', 'gama_UK.tif', 'gama_UTh.tif']

In [ ]:
# Populate Dataframe for NON-IRON MINERALIZATIONS
data_nomin = pd.DataFrame(columns = df_columns, dtype='float64')

for idx, point in enumerate(nomin):
    columns_index = 0
    while columns_index < len(df_columns):
        for file in files:
            ds = gdal.Open("inputs/" + file)
            r, g, b = getRGBValue(ds, point[0], point[1])
            data_nomin.at[idx, df_columns[columns_index]] = int(r) 
            data_nomin.at[idx, df_columns[columns_index+1]] = int(g)
            data_nomin.at[idx, df_columns[columns_index+2]] = int(b) 
            columns_index += 3


# Training labels
#data['labels'] = np.repeat(int(1), len(points_list), axis=None)
data_nomin['labels'] = np.repeat(int(0), len(nomin), axis=None)

In [ ]:
# POPULATE DATAFRAME FOR NON-MINERALIZATIONS
data_min = pd.DataFrame(columns = df_columns, dtype='float64')

for idx, point in enumerate(points_list):
    columns_index = 0
    while columns_index < len(df_columns):
        for file in files:
            ds = gdal.Open("inputs/" + file)
            r, g, b = getRGBValue(ds, point[0], point[1])
            data_min.at[idx, df_columns[columns_index]] = int(r) 
            data_min.at[idx, df_columns[columns_index+1]] = int(g)
            data_min.at[idx, df_columns[columns_index+2]] = int(b) 
            columns_index += 3
data_min['labels'] = np.repeat(int(1), len(points_list), axis=None)


In [ ]:
# REMOVE DUPLICATES, RANDOMIZE
data_nomin = data_nomin.drop_duplicates()
data_min = data_min.drop_duplicates()
data = data_min.append(data_nomin)
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
# NORMALIZE TRAINING DATASET
nn_dataset = data

stats = nn_dataset.describe(include = 'all')
stats.pop('labels')
stats = stats.transpose()
labels = nn_dataset.pop('labels')
normed_data = norm(nn_dataset, stats).fillna(0)

In [ ]:
# BUILD AND FIT MODEL
model = build_model(normed_data)
fitted_model = model.fit(normed_data, labels,
  epochs=EPOCHS, verbose=0,
  callbacks=[PrintDot()])


In [ ]:
# Loss Graphic
plt.plot(fitted_model.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
# This can seriously takes few hours of processing. Gotta optimize via Cython or vectorizing.
image_data = pd.DataFrame(columns = df_columns, dtype='float64')

columns_index = 0
for file in files:
    idx = 0
    opened_file = gdal.Open("inputs/" + file)
    r_band = opened_file.GetRasterBand(1).ReadAsArray()
    g_band = opened_file.GetRasterBand(2).ReadAsArray()
    b_band = opened_file.GetRasterBand(3).ReadAsArray()
    for x in range(0, r_band.shape[0]):
        for y in range(0, r_band.shape[1]):
            image_data.at[idx, df_columns[columns_index]] = int(r_band[x, y])
            image_data.at[idx, df_columns[columns_index+1]] = int(g_band[x, y])
            image_data.at[idx, df_columns[columns_index+2]] = int(b_band[x, y])
            idx += 1
    columns_index += 3
        
# Save
image_data.to_pickle("inputs/image.pkl")

In [ ]:
# Read from pickle (for those who already have it)
image_data = pd.read_pickle("inputs/image.pkl")

In [ ]:
# Normalize image data
stats_image = image_data.describe()
pred = norm(image_data, stats)

In [ ]:
results = model.predict(pred)
np.savetxt('outputs/data.csv', results, delimiter=',') # Save predictions results as CSV

In [ ]:
# Check unique counts for pred
results_round = np.round(results, 1)
unique, counts = np.unique(results_round, return_counts=True)
dict(zip(unique, counts))

In [ ]:
# Plot in image
testfile = gdal.Open('inputs/mag_GT.tif') # Could be any other image
testband = testfile.GetRasterBand(1).ReadAsArray()
binary = np.zeros(testband.shape, dtype=np.uint8) #Could be r or b or g, whatever

width = binary.shape[0]
heigth = binary.shape[1]

for x in range(0, binary.shape[0]):
    for y in range(0, binary.shape[1]):
        if(results_round[x + (y * width)] > 0.9): 
            binary[x, y] = 255
        else:
            binary[x, y] = 0

# Save jpg for analysis
plt.imsave('outputs/ferro3.jpg', binary, dpi=300) 


In [ ]:
# Gdal Stuff
dst_ds = gdal.GetDriverByName('GTiff').Create("outputs/ferro.tiff", binary.shape[1], binary.shape[0], 1, gdal.GDT_Byte, options = ['PROFILE=GeoTIFF'])
file = gdal.Open('mag_GT.tif')
gdalnumeric.CopyDatasetInfo(file, dst_ds) # Copy proj info from original to new

dst_ds.GetRasterBand(1).WriteArray(binary)
dst_ds.GetRasterBand(1).SetNoDataValue(0)
dst_ds.FlushCache()


In [ ]:
# Transform points into shapefile 
src_ds = dst_ds
proj = osr.SpatialReference(wkt=src_ds.GetProjection())
srcband = src_ds.GetRasterBand(1)
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource( "outputs/ferro2.shp" )
dst_layer = dst_ds.CreateLayer("ProbOver90", proj )
gdal.Polygonize( srcband, None, dst_layer, -1, [], callback=None )
dst_ds.FlushCache()
